In [92]:
import time

class Monkey:
    def __init__(self, no, items, operation, divider, next_monkey):
        self.no = no
        self.items = items
        self.operation = operation
        self.divider = divider
        self.next_monkey = next_monkey
        self.activitiy = 0
        
    def inspect_items(self):
        self.activitiy += len(self.items)
        for idx in range(len(self.items)):
            exec(self.operation.replace("idx", str(idx)))
            
    def inspect_remainers(self):
        self.activitiy += len(self.remainers)
        for idx in range(len(self.remainers)):
            for no in self.remainers[0].keys():
                operation = self.operation.replace("idx", str(idx)).replace("no", f"'{no}'")
                exec(operation)
            
    def get_destination(self, remainer):
        return str(self.next_monkey[not remainer[self.no]])
        
        
class Hord(dict):
    def __init__(self, filename="d11.txt"):
        with open(filename, "r") as f:
            s = f.read() 
            for block in s.split("\n\n"):
                lines = block.split("\n")
                no = lines[0].replace("Monkey", "").strip(": ")
                items = [int(item) for item in lines[1].replace("Starting items:", "").split(",")]
                operation = lines[2].replace("Operation:", "").strip()
                operation = operation.replace("old", "new").replace("new", "self.remainers[idx][no]")
                divider = int(lines[3].split(" ")[-1])
                next_monkey = {True: int(lines[4].split(" ")[-1]),
                               False: int(lines[5].split(" ")[-1])}
                self[no] = Monkey(no, items, operation, divider, next_monkey)
            
        self._create_remainers()
        self.dividers = {no: monkey.divider for no, monkey in self.items()}
    
    def _create_remainers(self):
        for monkey in self.values():
            monkey.remainers = []
            for item in monkey.items:
                remainer = dict()
                for no in self.keys():
                    remainer[no] = item % self[no].divider
                monkey.remainers.append(remainer)
    
    def __repr__(self):
        s = ""
        for no, monkey in self.items():
            s1 = ""
            for remainer in monkey.remainers:
                s1 += str(list(remainer.values())) + "  "
            s += f"Monkey {no} ({len(monkey.remainers)}) {s1[:-2]}\n"
        return s
    
    
    def do_round(self):
        for no, monkey in self.items():
            #s = f"Within no={no}, before={len(monkey.remainers)}, destination: "
            monkey.inspect_remainers()
            for remainer in monkey.remainers:
                for no2 in remainer.keys():
                    remainer[no2] = remainer[no2] % self.dividers[no2]
            
            for remainer in monkey.remainers:
                destination = monkey.get_destination(remainer)
                #s += f"{destination} "
                self[destination].remainers.append(remainer)
            
            monkey.remainers = list()
            #print(s)
    
    def get_activitiy(self):
        return [monkey.activitiy for monkey in self.values()]
    
hord = Hord()
hord

Monkey 0 (2) [0, 10, 1, 0, 2, 2, 14, 8]  [3, 1, 0, 0, 1, 0, 10, 2]
Monkey 1 (8) [4, 10, 0, 2, 5, 0, 3, 16]  [3, 1, 0, 0, 1, 0, 10, 2]  [1, 9, 0, 8, 2, 2, 1, 10]  [4, 2, 1, 1, 2, 1, 11, 3]  [3, 7, 1, 8, 3, 1, 5, 16]  [4, 9, 0, 12, 1, 1, 13, 7]  [0, 8, 1, 7, 1, 1, 0, 9]  [3, 0, 0, 10, 4, 1, 3, 12]
Monkey 2 (5) [4, 3, 1, 4, 6, 0, 1, 12]  [2, 9, 1, 6, 6, 1, 12, 2]  [2, 0, 1, 12, 0, 2, 9, 1]  [3, 0, 0, 10, 4, 1, 3, 12]  [2, 10, 1, 9, 3, 0, 2, 11]
Monkey 3 (1) [4, 0, 1, 8, 1, 0, 14, 4]
Monkey 4 (3) [0, 5, 0, 8, 4, 0, 9, 3]  [2, 2, 1, 5, 1, 0, 6, 0]  [2, 8, 0, 0, 3, 1, 1, 14]
Monkey 5 (6) [1, 3, 1, 0, 0, 1, 6, 15]  [2, 5, 0, 4, 5, 1, 14, 6]  [0, 8, 1, 7, 1, 1, 0, 9]  [3, 7, 1, 8, 3, 1, 5, 16]  [4, 7, 0, 6, 0, 0, 16, 8]  [3, 9, 1, 1, 4, 2, 2, 15]
Monkey 6 (4) [3, 0, 0, 10, 4, 1, 3, 12]  [4, 8, 0, 9, 4, 2, 6, 17]  [3, 2, 0, 3, 5, 2, 0, 11]  [1, 1, 0, 4, 0, 2, 5, 18]
Monkey 7 (7) [4, 10, 0, 2, 5, 0, 3, 16]  [2, 5, 0, 4, 5, 1, 14, 6]  [2, 6, 0, 7, 2, 0, 4, 15]  [1, 5, 1, 6, 1, 2, 3, 14]  [3, 9, 1

In [93]:
t0 = time.time()
for round in range(10_000):
    hord.do_round()
print(f"Computation time {time.time()-t0:.1f}s")
sorted(hord.get_activitiy())

Computation time 283.8s


[49887, 55635, 130660, 130983, 132374, 132736, 140002, 168658]

In [94]:
140002* 168658

23612457316